In [1]:
import numpy as np

In [35]:
def compute_error_decrease(uc,uf,VXc,VXf,Old2New):
    
    N = len(VXc)-1
    err = np.zeros(N)

    for n, triple in enumerate(Old2New):

        i = triple[0]
        j = triple[2]
        k = triple[1]

        xi   = VXc[i]
        xj = VXc[j]
        xk  =  xi + (xj - xi)/2

        uci  = uc[i]
        ucj =  uc[j]

        ufi = uf[i]
        ufk = uf[k]
        ufj = uf[j]

        a  = (ucj - uci) / (xj - xi)
        a1 = (ufk - ufi) / (xk - xi)
        a2 = (ufj - ufk) / (xj - xk)

        b = uci - a * xi
        b1 = ufi - a1 * xi
        b2 = ufj - a2 * xj

        int1 = ((a-a1)**2/3) * (xk**3 - xi**3) + (b-b1)**2 * (xk - xi) + (a-a1)*(b-b1) * (xk**2 - xi**2)
        int2 = ((a-a2)**2/3) * (xj**3 - xk**3) + (b-b2)**2 * (xj - xk) + (a-a2)*(b-b2) * (xj**2 - xk**2)
        
        err[n] = np.sqrt(int1 + int2)

    return err

Test case

In [36]:
uc = np.array([0,0])
uf = np.array([0,0,1/2])

VXc = np.array([0,1])
VXf = np.array([0,1/2,1])

Old2New = np.array([[0,2,1]])

compute_error_decrease(uc,uf,VXc,VXf,Old2New)


array([0.28867513])

*c)* Same as in 1.6.

In [ ]:
def refine_marked(EToVcoarse, xcoarse, idxMarked):
    N = len(EToVcoarse[:,0]) + 1

    # ETofine = EToVcoarse.copy()
    # xcoarse = xcoarse.copy()

    for idx in idxMarked:
        xi   = xcoarse[EToVcoarse[idx][0]]
        xip1 = xcoarse[EToVcoarse[idx][1]]
        xih  =  xi + (xip1 - xi)/2

        xcoarse = np.hstack((xcoarse,[xih]))

        M = EToVcoarse[idx][1]
        EToVcoarse[idx][1] = N 
        EToVcoarse = np.vstack((EToVcoarse,[N,M]))

        N += 1

    return EToVcoarse, xcoarse

*d)*

We obtain the weak formulation
\begin{equation}
    -\int_{\Omega} v'u'\,dx - \int_{\Omega} vu dx = \int_{\Omega} vf dx
\end{equation}
and see that the left hand side is the same is in problem 1.1. Thus the elemental contributions to $A$ are as before up to multiplication with $-1$.

In the exercise it is given that
$$\int_{0}^L f(x)v(x) dx \approx \sum_{j=1}^M \hat{f}_j \int_0^L N_i(x)N_j(x) dx$$ 
which we may reduce since $N_i, N_j$ only overlap for $j=i-1,i,i+1$. We note that these integrals were part of the computation of the $K_i$ matrix earlier. Thus the elemental contribution to the approximation of $f is


*f)*

In [ ]:
def AMR(u,VX,EToV,tol=10e-4, alpha=1):
    idxMarked = [0]


    while len(idxMarked)>0:
        err = compute_error_decrease(u,VX,EToV)
        idxMarked = np.where(err > tol*alpha)[0]
        EToV, VX = refine_marked(EToV,VX,idxMarked)
    
    return EToV, VX, err

Assembly

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
K = lambda h, psi, eps: np.array([[eps/h + psi/2, -eps/h + psi/2], [-eps/h - psi/2, eps/h - psi/2]])

def GlobalAssembly(x,eps,psi,c,d):
    M = len(x)
    nnzmax = 4 * M
    ii = np.ones(nnzmax, dtype=int)
    jj = np.ones(nnzmax, dtype=int)
    ss = np.zeros(nnzmax)
    b = np.zeros(M)
    count = 0

    for i in range(M - 1):
        j
        k
        h = x[i+1] - x[i]
        b[i] += h/2
        b[i+1] += h/2

        Ki = K(h,psi,eps)

        ii[count:count + 4] = [i, i, i + 1, i + 1]
        jj[count:count + 4] = [i, i + 1, i + 1, i]
        ss[count:count + 4] = [
        Ki[0, 0],
        Ki[0, 1],
        Ki[1, 1],
        Ki[1, 0]
        ]
        count += 4
    
    A = csr_matrix((ss[:count], (ii[:count], jj[:count])), shape=(M, M))

    # Boundary conditions
    b[0] = c
    b[1] -= A[0,1]*c

    A[0,0] = 1
    A[0,1] = 0
    A[1,0] = 0
    
    b[M-1] = d
    b[M-2] -= A[M-1,M-2]*d

    A[M-1,M-1] = 1
    A[M-1,M-2] = 0
    A[M-2,M-1] = 0

    return A, b

In [ ]:
from scipy import sparse
import numpy as np

def BVP1D(L, x, eps,psi=1,c=0, d=0, plot=True):
    
    if type(x) == int:
        x = np.linspace(0, L, x)

    A,b = GlobalAssembly(x,eps,psi,c,d)

    u = sparse.linalg.spsolve(A, b)
    
    if plot:
        plt.plot(x, u, 'r--',label="FEM solution")
        plt.show()

    return u
